In [0]:
import nltk
#nltk.download()
import pandas as pd
import sklearn.preprocessing as preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split 
from sklearn.svm import LinearSVC
import sklearn.metrics as metrics
import numpy as np
from sklearn.model_selection import learning_curve
import re
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import StratifiedShuffleSplit


In [0]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1abzhIX8Mhkr7igA0vA32dd01btuyeipk'}) # replace the id with id of file you want to access
downloaded.GetContentFile('SMSSpamCollection.txt') 

In [0]:
#Read file as panda dataframe
import pandas as pd
data = pd.read_table('SMSSpamCollection.txt') 
print(data.head())

    ham Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
0   ham                      Ok lar... Joking wif u oni...                                                             
1  spam  Free entry in 2 a wkly comp to win FA Cup fina...                                                             
2   ham  U dun say so early hor... U c already then say...                                                             
3   ham  Nah I don't think he goes to usf, he lives aro...                                                             
4  spam  FreeMsg Hey there darling it's been 3 week's n...                                                             


In [0]:
#data.info()
#print(data)
data.columns
y = data.iloc[:,0]
#print(y)
#y.value_counts()
raw_text = data.iloc[:,1]
#print(rawtext)
labelencoder = preprocessing.LabelEncoder()
y_trans = labelencoder.fit_transform(y)
print(y_trans)

[0 1 0 ... 0 0 0]


In [0]:
stop_words = nltk.corpus.stopwords.words('english')
porter = PorterStemmer()

Preprocessing

In [0]:
def preprocess_text(messy_string):
    assert(type(messy_string) == str)
    cleaned = re.sub(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', messy_string)
    cleaned = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr',
                     cleaned)
    cleaned = re.sub(r'£|\$', 'moneysymb', cleaned)
    cleaned = re.sub(
        r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b',
        'phonenumbr', cleaned)
    cleaned = re.sub(r'\d+(\.\d+)?', 'numbr', cleaned)
    cleaned = re.sub(r'[^\w\d\s]', ' ', cleaned)
    cleaned = re.sub(r'\s+', ' ', cleaned)
    cleaned = re.sub(r'^\s+|\s+?$', '', cleaned.lower())
    return ' '.join(
        porter.stem(term) 
        for term in cleaned.split()
        if term not in set(stop_words)
    )

In [0]:
result = raw_text.apply(preprocess_text)
#result = [result]
#print(result)

In [0]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_ngrams = vectorizer.fit_transform(result)
print(X_ngrams.shape)

(5571, 36333)


In [0]:
x_train,x_test,y_train,y_test = train_test_split(X_ngrams,
    y_trans,
    test_size=0.2,
    random_state=42,
    stratify=y_trans)

In [0]:
clf = LinearSVC(loss='hinge')
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
metrics.f1_score(y_test, y_pred)

0.9263157894736842

In [0]:
pd.DataFrame(
    metrics.confusion_matrix(y_test, y_pred),
    index=[['actual', 'actual'], ['spam', 'ham']],
    columns=[['predicted', 'predicted'], ['spam', 'ham']]
)

predicted     
                 spam  ham
actual spam       962    3
       ham         18  132

In [0]:
sample_space = np.linspace(500, len(raw_text) * 0.8, 10, dtype='int')

train_sizes, train_scores, valid_scores = learning_curve(
    estimator=LinearSVC(loss='hinge', C=1e10),
    X=X_ngrams,
    y=y_trans,
    train_sizes=sample_space,
    cv=StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=40),
    scoring='f1',
    n_jobs=-1
)

In [0]:
def spam_filter(message):
    if clf.predict(vectorizer.transform([preprocess_text(message)])):
        return 'spam'
    else:
        return 'not spam'

In [0]:
spam_filter('call me tonight to 7036930677')

'spam'